This notebook measures the intensities and errors of CO 2-1 emission at the locations of all of the SNe in our sample across all of the resolutions in the survey. This is used to prepare to answer the question in section 3.1 Do we see gas at supernova locations? 

We will use this to uncover what resolution will be best for this study.

In [2]:
# IMPORT PACKAGES

#global
import numpy as np
from matplotlib import pyplot as plt
import astropy
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
from astropy.io import fits
import os
from reproject import reproject_interp

#local
import sys
sys.path.append('/home/maykerchen.1/Desktop/PythonFunctions')

from findResolution import findRes
from getMapValue    import getValue
from nonZeroError   import findErrVals
from nonZeroError   import nonZeroErrArray
from nonZeroError   import findSignal

In [3]:
# DEFINE FUNCTIONS

def genFileName(galaxy, mapType, res, telOrient):
    
    """
    Generates the filenames for the CO maps.
    
    Parameters
    ----------
    galaxy    : string : name of galaxy
    mapType   : string : "int", "err", or "EW"   
    res       : string : resolution of map ("" (native), "_60pc", "_90pc", "_120pc", "_150pc", etc.)
    telOrient : string : 12m+7m+tp, 7m+tp, etc.
    
    Returns
    -------
    fileName : string : full path to file on Tycho.
        (/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_150pc_broad_mom0.fits)

    """
    
    if mapType == "int":
        mapTypeStr = "_broad_mom0"
    elif mapType == "err":
        mapTypeStr = "_broad_emom0"
    elif mapType  == "EW":
        mapTypeStr = "_strict_ew"
    else:
        print("Wrong Map Type.")
        
    preamble = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" 
    
    fileName = preamble + galaxy + "/" + galaxy + "_" + telOrient + "_co21" + res + mapTypeStr + ".fits"
    return(fileName)


def getAlphaCO(acofileName, intfileName, SNra, SNdec):
    """
    Pulls the metallicity dependent Alpha CO value at the SN location using Sun+2020 maps
    or assigns a value of alpha CO = 4.35/0.65 if no Sun value is available.
    """

    if(os.path.isfile(acofileName) == True) and (os.path.isfile(intfileName) == True):
        hdu_int = pyfits.open(intfileName)
        hdu_aco = pyfits.open(acofileName)
        acoMap, footprint = reproject_interp(hdu_aco, hdu_int[0].header)

        new_hdu = fits.PrimaryHDU(acoMap, hdu_int[0].header)
        outco = "../Data/ACOrewrites/temp.fits"
        new_hdu.writeto(outco, overwrite=True)
        
        alphaCO = getValue(outco,float(SNra),float(SNdec))
        if np.isfinite(alphaCO) == False:
            print(intfileName)
            alphaCO = 6.7
    else:
        print("NO ACO file: ", acofileName)
        alphaCO = 4.35/0.65
    
    return(alphaCO)

def findSNRErr(errorArray, i, res):
    """
    Finds the lowest non-zero error for use in calculating the SNR in the map if the pixel has a non-real error associated with it.
    Otherwise just uses the pixel error.
    """
    
    if(errorArray[i] <= 0.0):   
        hdulist = pyfits.open(genFileName(galaxies[i], "err", res, telOrient[i]))
        map = hdulist[0].data 
        flatMap = map.flatten()
        flatMap = flatMap[flatMap > 0.0]
        flatMap.sort()
        usedErr = flatMap[0]
    else: 
        usedErr = errorArray[i]
    
    return(usedErr)
    

In [4]:
# import SNeCO data

dataFile = '../Data/3.SNe+GalData.csv'
dataTable = Table.read(dataFile, format='csv') 
dataTable.colnames
print(dataTable['orient_incl'])


orient_incl
-----------
       64.3
       75.0
        8.9
       34.7
       42.9
       48.6
       48.6
       55.4
       55.4
       55.4
        ...
       90.0
      45.33
      45.33
       24.0
       24.0
       24.0
       24.0
       24.0
       61.9
       52.7
       50.0
Length = 63 rows


In [19]:
# pull galaxy names and telescope orientation to generate the file lists

galaxies  = dataTable['galaxy']
telOrient = dataTable['telOrient']
SNname    = dataTable['SN_name']
SNra      = dataTable['SN_ra']
SNdec     = dataTable['SN_dec']
beamsize  = dataTable['map_beamsize']
telOrient = dataTable['telOrient']
distance  = dataTable['dist']
acoFile   = dataTable['AlphaCOFile']
incl      = dataTable["orient_incl"]

incl_rad = []
for i in range(len(incl)):
    if incl[i] == 90.0:
        incl[i] = 85
    incl_rad.append(incl[i] * np.pi/180)
    
    
cd1, cd2, dist_mpc, ppb = [],[],[],[]
for i in range(len(galaxies)):
    filen = genFileName(galaxies[i], "int", "_150pc", telOrient[i])
    if os.path.isfile(filen):
        hdu_int = pyfits.open(filen)

        cdelt1, cdelt2, distance_mpc = np.abs(hdu_int[0].header['CDELT1']), np.abs(hdu_int[0].header['CDELT2']), distance[i]
        pixelArea = cdelt1*cdelt2 * (np.pi/180.0 * distance_mpc * 10**6)**2
        beamArea = (150.0/2.0)**2*np.pi/np.log(2.0)
        pixelsPerBeam = beamArea/pixelArea
        ppb.append(pixelsPerBeam)
        print(galaxies[i] + ": pixels per beam", pixelsPerBeam)
        cd1.append(cdelt1)
        cd2.append(cdelt2)
        dist_mpc.append(distance_mpc)
    else:
        cd1.append(float('nan'))
        cd2.append(float('nan'))
        dist_mpc.append(float('nan'))
        ppb.append(float('nan'))

    
        

circinus: pixels per beam 15.372292257453731
ngc0253: pixels per beam 19.807687028596337
ngc0628: pixels per beam 280.0573767903157
ngc1087: pixels per beam 47.97292093600939
ngc1097: pixels per beam 23.52651806858686
ngc1097: pixels per beam 23.52651806858686
ngc1365: pixels per beam 31.46828277001683
ngc1365: pixels per beam 31.46828277001683
ngc1365: pixels per beam 31.46828277001683
ngc1433: pixels per beam 34.72393907473135
ngc1511: pixels per beam 51.618809833571845
ngc1559: pixels per beam 31.890562101619572
ngc1559: pixels per beam 31.890562101619572
ngc1559: pixels per beam 31.890562101619572
ngc1559: pixels per beam 31.890562101619572
ngc1566: pixels per beam 38.51226218338703
ngc1566: pixels per beam 38.51226218338703
ngc1637: pixels per beam 88.04059558657772
ngc2775: pixels per beam 22.48809693537148
ngc2997: pixels per beam 60.96548793044351
ngc2997: pixels per beam 60.96548793044351
ngc3239: pixels per beam 102.18685776318237
ngc3627: pixels per beam 52.90349552950106
ng

In [20]:
#Create Intensity Measurement Database 

intenData = Table({'galaxy'  : galaxies,
              'SN_name'      : SNname,
              'ppb150'       : ppb,
              'dist_mpc'     : dist_mpc,
              'incl'         : incl,
              'NatRes'       : np.zeros(len(galaxies)),
              'NatInt'       : np.zeros(len(galaxies)),
              'pc60Int'      : np.zeros(len(galaxies)),
              'pc90Int'      : np.zeros(len(galaxies)),
              'pc120Int'     : np.zeros(len(galaxies)),
              'pc150Int'     : np.zeros(len(galaxies)),
              'pc500Int'     : np.zeros(len(galaxies)),
              'pc750Int'     : np.zeros(len(galaxies)),
              'pc1000Int'    : np.zeros(len(galaxies)),
              'NatErr'       : np.zeros(len(galaxies)),
              'pc60Err'      : np.zeros(len(galaxies)),
              'pc90Err'      : np.zeros(len(galaxies)),
              'pc120Err'     : np.zeros(len(galaxies)),
              'pc150Err'     : np.zeros(len(galaxies)),
              'pc500Err'     : np.zeros(len(galaxies)),
              'pc750Err'     : np.zeros(len(galaxies)),
              'pc1000Err'    : np.zeros(len(galaxies)),
              'NatEW'        : np.zeros(len(galaxies)),
              'pc60EW'       : np.zeros(len(galaxies)),
              'pc90EW'       : np.zeros(len(galaxies)),
              'pc120EW'      : np.zeros(len(galaxies)),
              'pc150EW'      : np.zeros(len(galaxies)),
              'pc500EW'      : np.zeros(len(galaxies)),
              'pc750EW'      : np.zeros(len(galaxies)),
              'pc1000EW'     : np.zeros(len(galaxies)),
              'NatSNR'       : np.zeros(len(galaxies)),
              'pc60SNR'      : np.zeros(len(galaxies)),
              'pc90SNR'      : np.zeros(len(galaxies)),
              'pc120SNR'     : np.zeros(len(galaxies)),
              'pc150SNR'     : np.zeros(len(galaxies)),
              'pc500SNR'     : np.zeros(len(galaxies)),
              'pc750SNR'     : np.zeros(len(galaxies)),
              'pc1000SNR'    : np.zeros(len(galaxies)),
              'alphaCO'      : np.zeros(len(galaxies)),
              'pc60mgsd'     : np.zeros(len(galaxies)),
              'pc90mgsd'     : np.zeros(len(galaxies)),
              'pc120mgsd'    : np.zeros(len(galaxies)),
              'pc150mgsd'    : np.zeros(len(galaxies)),
              'pc500mgsd'    : np.zeros(len(galaxies)),
              'pc750mgsd'    : np.zeros(len(galaxies)),
              'pc1000mgsd'   : np.zeros(len(galaxies)),
              'pc60mgsdErr'  : np.zeros(len(galaxies)),
              'pc90mgsdErr'  : np.zeros(len(galaxies)),
              'pc120mgsdErr' : np.zeros(len(galaxies)),
              'pc150mgsdErr' : np.zeros(len(galaxies)),
              'pc500mgsdErr' : np.zeros(len(galaxies)),
              'pc750mgsdErr' : np.zeros(len(galaxies)),
              'pc1000mgsdErr': np.zeros(len(galaxies)),
                })


In [21]:
# populate database

for i in range(len(galaxies)):
    
    intfileName = genFileName(galaxies[i], "int", "_150pc", telOrient[i])
    
    intenData['NatRes'][i]    = findRes(float(beamsize[i]),float(distance[i]))
    
    intenData['NatInt'][i]    = getValue(genFileName(galaxies[i], "int", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60Int'][i]   = getValue(genFileName(galaxies[i], "int", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90Int'][i]   = getValue(genFileName(galaxies[i], "int", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120Int'][i]  = getValue(genFileName(galaxies[i], "int", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150Int'][i]  = getValue(genFileName(galaxies[i], "int", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500Int'][i]  = getValue(genFileName(galaxies[i], "int", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750Int'][i]  = getValue(genFileName(galaxies[i], "int", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000Int'][i] = getValue(genFileName(galaxies[i], "int", "_1kpc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    
    intenData['NatErr'][i]    = getValue(genFileName(galaxies[i], "err", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60Err'][i]   = getValue(genFileName(galaxies[i], "err", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90Err'][i]   = getValue(genFileName(galaxies[i], "err", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120Err'][i]  = getValue(genFileName(galaxies[i], "err", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150Err'][i]  = getValue(genFileName(galaxies[i], "err", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500Err'][i]  = getValue(genFileName(galaxies[i], "err", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750Err'][i]  = getValue(genFileName(galaxies[i], "err", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000Err'][i] = getValue(genFileName(galaxies[i], "err", "_1kpc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
        
    
    intenData['NatEW'][i]     = getValue(genFileName(galaxies[i], "EW", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60EW'][i]    = getValue(genFileName(galaxies[i], "EW", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90EW'][i]    = getValue(genFileName(galaxies[i], "EW", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000EW'][i]  = getValue(genFileName(galaxies[i], "EW", "_1kpc", telOrient[i]),float(SNra[i]),float(SNdec[i]))

    intenData['NatSNR'][i]    = findSignal(intenData['NatInt'][i],    findSNRErr(intenData['NatErr'], i, ""))
    intenData['pc60SNR'][i]   = findSignal(intenData['pc60Int'][i],   findSNRErr(intenData['pc60Err'], i, "_60pc"))
    intenData['pc90SNR'][i]   = findSignal(intenData['pc90Int'][i],   findSNRErr(intenData['pc90Err'], i, "_90pc"))
    intenData['pc120SNR'][i]  = findSignal(intenData['pc120Int'][i],  findSNRErr(intenData['pc120Err'], i, "_120pc"))
    intenData['pc150SNR'][i]  = findSignal(intenData['pc150Int'][i],  findSNRErr(intenData['pc150Err'], i, "_150pc"))
    intenData['pc500SNR'][i]  = findSignal(intenData['pc500Int'][i],  findSNRErr(intenData['pc500Err'], i, "_500pc"))
    intenData['pc750SNR'][i]  = findSignal(intenData['pc750Int'][i],  findSNRErr(intenData['pc750Err'], i, "_750pc"))
    intenData['pc1000SNR'][i] = findSignal(intenData['pc1000Int'][i], findSNRErr(intenData['pc1000Err'], i, "_1kpc"))
        
    intenData['alphaCO'][i]   = getAlphaCO(acoFile[i], intfileName, float(SNra[i]), float(SNdec[i])) 
    
    intenData['pc60mgsd'][i]   = intenData['pc60Int'][i]   * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc90mgsd'][i]   = intenData['pc90Int'][i]   * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc120mgsd'][i]  = intenData['pc120Int'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc150mgsd'][i]  = intenData['pc150Int'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc500mgsd'][i]  = intenData['pc500Int'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc750mgsd'][i]  = intenData['pc750Int'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc1000mgsd'][i] = intenData['pc1000Int'][i] * intenData['alphaCO'][i] * np.cos(incl_rad[i])
   
    intenData['pc60mgsdErr'][i]   = intenData['pc60Err'][i]   * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc90mgsdErr'][i]   = intenData['pc90Err'][i]   * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc120mgsdErr'][i]  = intenData['pc120Err'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc150mgsdErr'][i]  = intenData['pc150Err'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc500mgsdErr'][i]  = intenData['pc500Err'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc750mgsdErr'][i]  = intenData['pc750Err'][i]  * intenData['alphaCO'][i] * np.cos(incl_rad[i])
    intenData['pc1000mgsdErr'][i] = intenData['pc1000Err'][i] * intenData['alphaCO'][i] * np.cos(incl_rad[i])
   


NO ACO file:  /data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1068_alphaCO21_scaling.fits
NO ACO file:  /data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1672_alphaCO21_scaling.fits
NO ACO file:  /data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC4579_alphaCO21_scaling.fits
NO ACO file:  /data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC4579_alphaCO21_scaling.fits
/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc4945/ngc4945_7m+tp_co21_150pc_broad_mom0.fits


In [22]:
intenData


galaxy,SN_name,ppb150,dist_mpc,incl,NatRes,NatInt,pc60Int,pc90Int,pc120Int,pc150Int,pc500Int,pc750Int,pc1000Int,NatErr,pc60Err,pc90Err,pc120Err,pc150Err,pc500Err,pc750Err,pc1000Err,NatEW,pc60EW,pc90EW,pc120EW,pc150EW,pc500EW,pc750EW,pc1000EW,NatSNR,pc60SNR,pc90SNR,pc120SNR,pc150SNR,pc500SNR,pc750SNR,pc1000SNR,alphaCO,pc60mgsd,pc90mgsd,pc120mgsd,pc150mgsd,pc500mgsd,pc750mgsd,pc1000mgsd,pc60mgsdErr,pc90mgsdErr,pc120mgsdErr,pc150mgsdErr,pc500mgsdErr,pc750mgsdErr,pc1000mgsdErr
str8,str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
circinus,SN1996cr,15.372292257453731,4.2,64.3,148.96539476404914,186.84803771972656,nan,nan,nan,186.84803771972656,111.39796447753906,97.68943786621094,81.1122817993164,2.1378190517425537,nan,nan,nan,2.1378190517425537,0.24664470553398132,0.17664189636707306,0.16147032380104065,34.56536865234375,nan,nan,nan,34.56536865234375,49.76144790649414,69.42247772216797,86.25480651855469,87.40124079604642,nan,nan,nan,87.40124079604642,451.65358095307374,553.036623107839,502.3355368956884,4.968773833170559,nan,nan,nan,402.6115402289611,240.03519974851744,210.4966984047925,174.77700651956354,nan,nan,nan,4.606473964923083,0.5314586441272052,0.3806198172227499,0.34792881188466773
ngc0253,SN1940E,19.807687028596337,3.7,75.0,150.1701284517627,219.3036651611328,nan,nan,nan,219.3036651611328,153.83908081054688,153.87240600585938,142.72422790527344,1.465811014175415,nan,nan,nan,1.465811014175415,0.18237826228141785,0.11332354694604874,0.09203742444515228,45.862266540527344,nan,nan,nan,45.862266540527344,60.678550720214844,65.11054229736328,69.24812316894531,149.61251009871899,nan,nan,nan,149.61251009871899,843.5165402177496,1357.8149480188367,1550.7194900953239,4.479999295527333,nan,nan,nan,254.2846041302592,178.3778202472373,178.41646111578535,165.49004671683653,nan,nan,nan,1.6996212680508755,0.21146926200307817,0.13139968843037825,0.10671823484121151
ngc0628,SN2013ej,280.0573767903157,9.84,8.9,53.490181864724654,5.771303653717041,4.944133281707764,2.7272579669952393,1.8072082996368408,1.341525912284851,0.6933073997497559,0.5306633114814758,0.5881704092025757,1.9949342012405396,1.7279061079025269,1.2605571746826172,1.0164546966552734,0.8571658730506897,0.18906767666339874,0.1100897341966629,0.06774332374334335,nan,nan,nan,nan,nan,nan,1.9705547094345093,4.776077747344971,2.892979452719887,2.861343714855754,2.1635337307741773,1.7779526284679545,1.5650715391996457,3.6669800570092472,4.820279705040487,8.68233763419928,7.274922328334363,35.535123822636834,19.60170651386112,12.989004754076928,9.641991161664595,4.9830299657189085,3.8140530214653197,4.227375584139824,12.419033630298495,9.060042020628465,7.305596643072226,6.160735097512134,1.3588920278402108,0.791251390967422,0.4868937102248143
ngc1068,SN2018ivc,nan,nan,34.7,588.6196760664847,127.7774887084961,nan,nan,nan,nan,nan,128.53480529785156,122.50003814697266,0.6721417903900146,nan,nan,nan,nan,nan,0.4782250225543976,0.3369501829147339,32.07529830932617,nan,nan,nan,nan,nan,36.46405792236328,44.54560470581055,190.10496079160973,nan,nan,nan,nan,nan,268.77473832568194,363.55533950836764,6.692307692307692,nan,nan,nan,nan,nan,707.2037545337462,674.0002189081329,nan,nan,nan,nan,nan,2.631213628703481,1.8539136842813995
ngc1087,SN1995V,47.97292093600939,15.85,42.9,123.1488071983115,15.466903686523438,nan,nan,15.466903686523438,12.753849029541016,4.536659240722656,4.371304512023926,4.203847885131836,1.1405595541000366,nan,nan,1.1405595541000366,0.9069606065750122,0.2801399230957031,0.1639837622642517,0.10871382057666779,6.007981300354004,nan,nan,6.007981300354004,6.287197589874268,8.803016662597656,14.90282154083252,20.871063

In [23]:
ascii.write(intenData, '../Data/3.IntenData.csv', format='csv', fast_writer=False, overwrite=True)  

In [39]:
# calculate sigmolPerPixel here

smpp, sm = [],[]
for i in range(len(intenData)):
    if intenData["pc150SNR"][i] >= 3.0:
        sigMol = intenData["pc150Int"][i] * intenData["alphaCO"][i] * np.cos(intenData["incl"][i]*np.pi/180)
        sigMolPerPix = intenData["pc150Int"][i] * intenData["alphaCO"][i] * np.cos(intenData["incl"][i]*np.pi/180) / intenData['ppb150'][i]
    else:
        sigMol = 3 * intenData["pc150Err"][i] * intenData["alphaCO"][i] * np.cos(intenData["incl"][i]*np.pi/180)
        sigMolPerPix = 3 * intenData["pc150Err"][i] * intenData["alphaCO"][i] * np.cos(intenData["incl"][i]*np.pi/180) / intenData['ppb150'][i]    
    smpp.append(np.round(sigMolPerPix,2))
    sm.append(np.round(sigMol,2))
    
SigMolVals = Table({"Galaxy":intenData["galaxy"],
                    "SN_name": intenData["SN_name"],
                    "SNType" : dataTable["SN_type"],
                    "Sigma Mol Per Beam" : sm,
                    "Sigma Mol Per Pixel": smpp,
                    "dist_mpc": intenData["dist_mpc"],
                    "incl": intenData["incl"],
                    "alphaCO": intenData["alphaCO"],
                    "pixPerBeam" : intenData["ppb150"]})
SigMolVals
ascii.write(S, '../Data/3.SigMolValues.csv', format='csv', fast_writer=False, overwrite=True)      
                    

Galaxy,SN_name,SNType,Sigma Mol Per Beam,Sigma Mol Per Pixel,dist_mpc,incl,alphaCO,pixPerBeam
str8,str11,str12,float64,float64,float64,float64,float64,float64
circinus,SN1996cr,II,402.61,26.19,4.2,64.3,4.968773833170559,15.372292257453731
ngc0253,SN1940E,I,254.28,12.84,3.7,75.0,4.479999295527333,19.807687028596337
ngc0628,SN2013ej,II,18.48,0.07,9.84,8.9,7.274922328334363,280.0573767903157
ngc1068,SN2018ivc,II,nan,nan,nan,34.7,6.692307692307692,nan
ngc1087,SN1995V,II,61.94,1.29,15.85,42.9,6.629858070576079,47.97292093600939
ngc1097,SN1999eu,II,11.57,0.49,13.58,48.6,10.912807259453894,23.52651806858686
ngc1097,SN1992bd,II,724.3,30.79,13.58,48.6,4.206428083333224,23.52651806858686
ngc1365,SN2001du,II,11.9,0.38,19.57,55.4,6.1035414066894775,31.46828277001683
ngc1365,SN1983V,Ic,15.63,0.5,19.57,55.4,5.226342740029572,31.46828277001683
